In [1]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("pretrain_models/vncorenlp/VnCoreNLP-1.1.1.jar", 
                    annotators="wseg", max_heap_size='-Xmx500m')
from tqdm import tqdm

In [2]:
# Load the model in fairseq
from fairseq.models.roberta import RobertaModel
# phoBERT = RobertaModel.from_pretrained('PhoBERT_base_fairseq', checkpoint_file='model.pt')
phoBERT = RobertaModel.from_pretrained('pretrain_models/PhoBERT_large_fairseq', checkpoint_file='model.pt')
phoBERT.eval()  # disable dropout (or leave in train mode to finetune

RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerSentenceEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(64001, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(258, 1024, padding_idx=1)
        (layers): ModuleList(
          (0): TransformerSentenceEncoderLayer(
            (dropout_module): FairseqDropout()
            (activation_dropout_module): FairseqDropout()
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwi

In [3]:
from fairseq.data.encoders.fastbpe import fastBPE  
from fairseq import options  
import numpy as np
import random

In [4]:
# Initialize Byte Pair Encoding for PhoBERT
class BPE():
    # bpe_codes = 'PhoBERT_base_fairseq/bpe.codes'
    bpe_codes = 'pretrain_models/PhoBERT_large_fairseq/bpe.codes'
args = BPE()
phoBERT.bpe = fastBPE(args) #Incorporate the BPE encoder into PhoBERT

In [5]:
# def fill_mask(text, n_sentences=1):
#     print('original sentence: ', text)
#     words_ = rdrsegmenter.tokenize(text)[0]
#     new_texts = []
#     remark = ''
#     for i in range(0, n_sentences):
#         words = words_.copy()
#         words[random.randint(2, len(words)-2)] = ' <mask>'
#         masked_text_tok = ' '.join(words)
#         print('======= {} ========'.format(i))
#         print('     masked_text_tok: ', masked_text_tok)
#         topk_filled_outputs = phoBERT.fill_mask(masked_text_tok, topk=1)
#         new_texts.append(topk_filled_outputs[0][0].replace('_', ' '))
#         print('     new text: ', new_texts[-1])
#     return new_texts

In [36]:
def fill_mask_sentence(text):
#     print('     original sentence: ', text)
    words_ = rdrsegmenter.tokenize(text)[0]
    while True:
        words = words_.copy()
        words[random.randint(0, len(words)-1)] = ' <mask>'
        masked_text_tok = ' '.join(words)
        topk_filled_outputs = phoBERT.fill_mask(masked_text_tok, topk=1)
        new_text = topk_filled_outputs[0][0]
        new_text = new_text.replace('_', ' ')
        new_text = new_text.replace(' ;', ';')
        new_text = new_text.replace(' .', '.')
        new_text = new_text.replace(' :', ':')
        new_text = new_text.replace(' ,', ',')
        new_text = new_text.replace('( ', '(')
        new_text = new_text.replace(' )', ')')
        new_text = new_text.replace(' ?', '?')
        if new_text != text:
#             print('     masked_text_tok: ', masked_text_tok)
#             print('     new text: ', new_text)
            return new_text

In [24]:
def fill_mask_para(text, n_max_random_sentences=10):
#     print('=== original para: ', text)
    sentences = text.rsplit('\n')
    if len(sentences) == 1:
        n_random_sentences = 1
    elif n_max_random_sentences < len(sentences):
        n_random_sentences = random.randint(1, n_max_random_sentences-1)
    else:
        n_random_sentences = random.randint(1, len(sentences)-1)
#     print('n_sentences: ', len(sentences))
#     print('n_random_sentences: ', n_random_sentences)
    random_ids = random.sample(list(range(0, len(sentences))), n_random_sentences)
    for random_id in random_ids:
        new_sentence = fill_mask_sentence(sentences[random_id])
        sentences[random_id] = new_sentence
    new_para = '\n'.join(sentences)
#     print('=== new para: ', new_para)
    return new_para

In [25]:
# test_para = '1. Phạm vi tuần tra:\na) Báo động lũ ở cấp I, bố trí người tuần tra như sau:\n- Lượt đi: 01 người (ban ngày), 02 người (ban đêm) kiểm tra mặt đê, mái đê phía sông, khu vực hành lang bảo vệ đê phía sông;\n- Lượt về: 01 người (ban ngày), 02 người (ban đêm) kiểm tra mái đê phía đồng, khu vực hành lang bảo vệ đê phía đồng, mặt ruộng, hồ ao gần chân đê phía đồng;\nb) Báo động lũ ở cấp II, bố trí người tuần tra như sau:\n- Lượt đi: 01 người kiểm tra mặt đê, mái đê phía sông, khu vực hành lang bảo vệ đê phía sông; 01 người (ban ngày), 02 người (ban đêm) kiểm tra mái đê phía đồng, khu vực hành lang bảo vệ đê phía đồng, mặt ruộng, hồ ao gần chân đê phía đồng;\n- Lượt về: 01 người (ban ngày), 02 người (ban đêm) kiểm tra mặt đê, mái đê phía sông, khu vực hành lang bảo vệ đê phía sông; 01 người kiểm tra mái đê phía đồng, khu vực hành lang bảo vệ đê phía đồng, mặt ruộng, hồ ao gần chân đê phía đồng;\nc) Báo động lũ ở cấp II và có tin bão khẩn cấp đổ bộ vào khu vực hoặc báo động lũ ở cấp III trở lên, bố trí người tuần tra như sau:\n- Lượt đi: 02 người kiểm tra mái đê, khu vực hành lang bảo vệ đê phía đồng, mặt ruộng, hồ ao gần chân đê phía đồng; 01 người kiểm tra mặt đê.\n- Lượt về: 02 người kiểm tra phía đồng; 01 người kiểm tra mặt đê, mái đê và khu vực hành lang bảo vệ đê phía sông.\nd) Mỗi kíp tuần tra phải kiểm tra vượt quá phạm vi phụ trách về hai phía, mỗi phía 50m. Đối với những khu vực đã từng xảy ra sự cố hư hỏng, phải kiểm tra quan sát rộng hơn để phát hiện sự cố.\n2. Người tuần tra, canh gác phải phát hiện kịp thời những hư hỏng của đê.\n3. Khi phát hiện có hư hỏng, người tuần tra phải tiến hành các công việc sau:\na) Xác định loại hư hỏng, vị trí, đặc điểm, kích thước của loại hư hỏng;\nb) Xác định mực nước sông so với mặt đê tại vị trí phát sinh hư hỏng;\nc) Đánh dấu bằng cách ghi bảng, cắm tiêu báo hiệu vị trí hư hỏng; nếu sự cố nghiêm trọng, phải cấm người, vật, xe cơ giới đi qua và bố trí người canh gác tại chỗ để theo dõi thường xuyên diễn biến của hư hỏng;\nd) Báo cáo kịp thời và cụ thể tình hình hư hỏng cho đội trưởng hoặc đội phó, cán bộ chuyên trách quản lý đê Điều và Ban Chỉ huy phòng, chống lụt, bão xã.'
test_para = 'Thông tư này hướng dẫn tuần tra, canh gác bảo vệ đê Điều trong mùa lũ đối với các tuyến đê sông được phân loại, phân cấp theo quy định tại Điều 4 của Luật Đê Điều.'
new_para = fill_mask_para(test_para)
print('=== original para: ', test_para)
print('=== new para: ', new_para)

=== original para:  Thông tư này hướng dẫn tuần tra, canh gác bảo vệ đê Điều trong mùa lũ đối với các tuyến đê sông được phân loại, phân cấp theo quy định tại Điều 4 của Luật Đê Điều.
=== new para:  Thông tư này quy định tuần tra, canh gác bảo vệ đê Điều trong mùa lũ đối với các tuyến đê sông được phân loại, phân cấp theo quy định tại Điều 4 của Luật Đê Điều.


In [26]:
################# Process corpus ####################

In [27]:
import os
import json

corpus_path = '/home/hana/sonnh/zalo-ai-2021/zac2021-ltr-data/new_corpus.json'
output_path = 'process_corpus'

In [28]:
with open(corpus_path, 'r') as f_corpus:
    corpus = json.load(f_corpus)
f_corpus.close()
n_laws = len(corpus)
print('n_laws: ', n_laws)

n_laws:  3261


In [30]:
new_corpus = []
for law in tqdm(corpus[:3]):
    if law['law_id'].find('nđ-cp') != -1:
        new_corpus.append(law)
        continue
    for idx, article in enumerate(law['articles']):
        text = article['text']
        if text == '':
            continue
#         print('=== original text: ', text)
        new_text = fill_mask_para(text, 5)
        law['articles'][idx]['article_id'] += '_syn'
        law['articles'][idx]['text'] = new_text
#         print('=== new text: ', new_text)
    new_corpus.append(law)
    
out_corpus_file = '/home/hana/sonnh/zalo-ai-2021/zac2021-ltr-data/syn_aug/syn_corpus.json'
with open(out_corpus_file, 'w', encoding='utf-8') as f_out:
    json.dump(new_corpus, f_out, ensure_ascii=False)
f_out.close()

100%|██████████| 3/3 [00:42<00:00, 14.26s/it]


In [31]:
################### Process training questions ###########################

In [38]:
question_path = '/home/hana/sonnh/zalo-ai-2021/zac2021-ltr-data/train_question_answer.json'
with open(question_path, 'r') as f_ques:
    ques = json.load(f_ques)
f_ques.close()
questions = ques['items']
n_questions = len(questions)
print('n_questions: ', n_questions)

n_questions:  3196


In [40]:
new_questions = []
for question in tqdm(questions):
    question['question_id'] += '_syn'
    question['question'] = fill_mask_para(question['question'], 5)
    for idx, relevant_article in enumerate(question['relevant_articles']):
        if relevant_article['law_id'].find('nđ-cp') != -1:
            continue
        question['relevant_articles'][idx]['article_id'] += '_syn'
    new_questions.append(question)
ques['items'] = new_questions

out_question_file = '/home/hana/sonnh/zalo-ai-2021/zac2021-ltr-data/syn_aug/syn_training_question.json'
with open(out_question_file, 'w', encoding='utf-8') as f_out:
    json.dump(ques, f_out, ensure_ascii=False)
f_out.close()

 23%|██▎       | 732/3196 [12:05<40:41,  1.01it/s]  


KeyboardInterrupt: 